In [1]:
import os
import joblib
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

In [2]:
model = joblib.load("KNN_Model")
le = joblib.load("KNN_Label_Encoder")

INPUT_DIR = "Custom_Input"

IMAGE_SIZE = (256, 256)
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'

In [3]:
def extract_color_features(image):
    features = []
    for i in range(3):
        features.append(np.mean(image[:, :, i]))
        features.append(np.std(image[:, :, i]))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for i in range(3):
        features.append(np.mean(hsv[:, :, i]))
        features.append(np.std(hsv[:, :, i]))
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    features.extend(hist)
    return features



def extract_texture_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, LBP_POINTS, LBP_RADIUS, method=LBP_METHOD)
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_POINTS + 3), range=(0, LBP_POINTS + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist.tolist()



def extract_shape_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return [0, 0, 0]
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    aspect_ratio = float(w) / h
    area = cv2.contourArea(c)
    rect_area = w * h
    extent = float(area) / rect_area if rect_area != 0 else 0
    hull = cv2.convexHull(c)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area if hull_area != 0 else 0
    return [aspect_ratio, extent, solidity]

In [4]:
def preprocess_and_classify_image(image_path, model, label_encoder):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    img = cv2.resize(img, IMAGE_SIZE)

    color_features = extract_color_features(img)
    texture_features = extract_texture_features(img)
    shape_features = extract_shape_features(img)

    features = np.hstack([color_features, texture_features, shape_features])

    features = features.reshape(1, -1)
    pred = model.predict(features)[0]
    pred_proba = model.predict_proba(features)[0]

    predicted_class = label_encoder.inverse_transform([pred])[0]

    return predicted_class, pred_proba

In [5]:
print(f"S No. \t\t Actual Class \t\t\t Predicted Class")
i=1
for folder in os.listdir(INPUT_DIR):
    folder_path = os.path.join(INPUT_DIR, folder)

    if not os.path.isdir(folder_path):
        continue
    for files in os.listdir(folder_path):
        image_path = os.path.join(folder_path, files)
        pred_class, prob = preprocess_and_classify_image(image_path, model, le)

        print(f"{i} \t\t {folder} \t\t\t {pred_class}")
        i+=1
        


S No. 		 Actual Class 			 Predicted Class
1 		 Pepper_BS 			 Tomato_Septoria_leaf_spot
2 		 Pepper_BS 			 Tomato_Septoria_leaf_spot
3 		 Pepper_Healthy 			 Tomato_Septoria_leaf_spot
4 		 Pepper_Healthy 			 Tomato_Septoria_leaf_spot
5 		 Potato_Healthy 			 Tomato_Septoria_leaf_spot
6 		 Potato_Healthy 			 Tomato_Septoria_leaf_spot
7 		 Potato_Healthy 			 Tomato_Septoria_leaf_spot
8 		 Potato_LB 			 Tomato_Septoria_leaf_spot
9 		 Potato_LB 			 Tomato_Septoria_leaf_spot
10 		 Tomato_Mosaic 			 Tomato_Septoria_leaf_spot
11 		 Tomato_Mosaic 			 Tomato_Septoria_leaf_spot
12 		 Tomato_Spider 			 Tomato_Septoria_leaf_spot
13 		 Tomato_Spider 			 Tomato_Septoria_leaf_spot


In [14]:
print(f"S No. \t\t Actual Class \t\t\t Predicted Class")
i=1
for folder in os.listdir(INPUT_DIR):
    folder_path = os.path.join(INPUT_DIR, folder)

    if not os.path.isdir(folder_path):
        continue
    for files in os.listdir(folder_path):
        image_path = os.path.join(folder_path, files)
        pred_class, prob = preprocess_and_classify_image(image_path, model, le)

        print(f"{i} \t\t {folder} \t\t\t {pred_class}")
        i+=1
        


S No. 		 Actual Class 			 Predicted Class
1 		 Pepper_BS 			 Pepper__bell___Bacterial_spot
2 		 Pepper_BS 			 Pepper__bell___Bacterial_spot
3 		 Pepper_Healthy 			 Pepper__bell___healthy
4 		 Pepper_Healthy 			 Pepper__bell___healthy
5 		 Potato_Healthy 			 Potato___healthy
6 		 Potato_Healthy 			 Tomato_Late_blight
7 		 Potato_Healthy 			 Potato___healthy
8 		 Potato_LB 			 Potato___Late_blight
9 		 Potato_LB 			 Potato___Late_blight
10 		 Tomato_Mosaic 			 Tomato__Tomato_mosaic_virus
11 		 Tomato_Mosaic 			 Tomato__Tomato_mosaic_virus
12 		 Tomato_Spider 			 Tomato_Spider_mites_Two_spotted_spider_mite
13 		 Tomato_Spider 			 Tomato_Spider_mites_Two_spotted_spider_mite
